In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## install & import Konlpy, Keybert, Transformers

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash
!pip install keybert
!pip install transformers

--2023-02-27 05:59:29--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-02-27 05:59:29--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb5902be309986c3698a2b3ffc7.dl.dropboxusercontent.com/cd/0/get/B3T2pkh1sqQPFIC6IUhGFzL8MaDDRomEwoVvXz7VLnb19Jot-WuW36rC5e1M2aeXJ1-UK3DwFxuW8eP1oRfaO_8nZAkBTY5OqhxAV3A6DQMd6VEer2EOL-rF2HKV_zJtdZluGmZNo0np-QAl_vsWZ-pSmQoG1IlEDB-Y_SqucdOcRLSpMGbCA_ARE6w6KX_RQXA/file?dl=1# [following]
--2023-02-27 05:59:29--  https://ucb5902be309986c3698a2b3ffc7.dl.dropboxusercontent.com/cd/0/get/B3T2pkh1sqQP

In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import pandas as pd
import re, string
from konlpy.tag import Okt, Mecab
from collections import Counter
from keybert import KeyBERT
from collections import Counter
from sentence_transformers import SentenceTransformer
from transformers import pipeline

## Crawling Naver_Finance_News

In [ ]:
# URL 페이지 조정 및 필터
now = datetime.now()
past_day = datetime.now() - timedelta(7)   # timedelta() 안에 들어가는 값은 day로 7일을 뜻함, 조정 가능
today = now.date().strftime("%Y-%m-%d") 
p_day = past_day.date().strftime("%Y-%m-%d")

In [ ]:
class NaverFinanceNewsCrawler:
    
    URL_NAVER_FINANCE = "http://finance.naver.com"
    URL_NAVER_FINANCE_NEWS_QUERY = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%s&x=0&y=0&stDateStart={p_day}&stDateEnd={today}&page=%s"

    def __init__(self):
        pass

    def crawl(self, query=None, page=1):  # 초기 설정
        """
        :param query: 기업 이름
        :param code: 기업 코드
        :param page: 크롤링 시작 페이지 (default 1p)
        """
        if query:  
            return self._crawl_by_query(query, page)   
        else:
            raise Exception("[오류] '기업 이름'은 필수입니다.")

    def _crawl_by_query(self, query, page):
        
        while page:
        # Convert the query to euc-kr string
            q = ""
            for c in query.encode('euc-kr'):
                q += "%%%s" % format(c, 'x').capitalize()  # capitalize : 맨 첫글자만 대문자로 변환

            r_url = NaverFinanceNewsCrawler.URL_NAVER_FINANCE_NEWS_QUERY % (q, page)
            r = requests.get(r_url)
            soup = BeautifulSoup(r.text, 'lxml') 
            news = soup.find('div', class_='newsSchResult').find('dl', class_='newsList')
            news_title = news.find_all('dt', class_='articleSubject')  # 모든 뉴스 타이틀
            news_title += news.find_all('dd', class_='articleSubject') # 태그가 2개 /dt /dd
            news_date = news.find_all('dd', class_='articleSummary')
            
            if len(news_title) == 0:  # 페이지 넘어가도 크롤링되는거 방지
                break
            
            for i, title in enumerate(news_title):
                url = NaverFinanceNewsCrawler.URL_NAVER_FINANCE + title.a.get("href")
                date = news_date[i].find('span', class_='wdate').text.strip(' \t\n\r')[:10]
                res = {                         # 마지막에 반환되는 딕셔너리 (title, url, content)
                    "title": title.a.text,
                    "date" : date
                }
                res.update(self._crawl_content(url))  # res dictionary 에다가 self._crawl_content(url)값을 업데이트 / .updata : 기존 dictionary의 데이터를 한꺼번에 바꿀때 유용한 함수
                yield res
            print(f'{query} 크롤링 중...현재 페이지 수 : {page}')
            page += 1
            
    def _crawl_content(self, url):
        global con
        try:
            r = requests.get(url)
            soup = BeautifulSoup(r.text, "lxml") #  r.text를 통해 가져온 HTML 문서를 lxml 파서를 통해서 BeautifulSoup 객체로 만들어 준 것 / lxml은 구문을 분석하기 위한 parser
            content = soup.find('div', id="content", class_='articleCont')
            content.find('span').decompose()   # 이미지 삭제
            content.find('div', class_='link_news').decompose()  # 링크 제거
            con = content.text.strip(' \t\n\r')
        except:
            pass
        return {"content": con}

In [ ]:
company_list = list(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/2.24_recommend.csv')['name'])
outputFile = '_%s.%s.%s' % (now.year, now.month, now.day)
company_list

['KCC', '금호타이어', '경동나비엔', '원풍', '서린바이오']

In [ ]:
# 크롤링
for company in company_list:
    company_outputFile = company+outputFile
    content = []
    title = []
    date = []
    crawler = NaverFinanceNewsCrawler()
    docs = crawler.crawl(query=company)

    for i, d in enumerate(docs):
        d["title"] = d["title"].replace(d["title"][d["title"].find('['):d["title"].find(']')+1], '').strip()  # [] 제거
        title.append(d["title"])
        content.append(d["content"])
        date.append(d['date'])
    
    output_df = pd.DataFrame({
        'date' : date,
        'title' : title,
        'content' : content,
        })

    output_df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/data/craw_data/{company_outputFile}.csv", mode='w',  encoding='UTF-8', index=False)

KCC 크롤링 중...현재 페이지 수 : 1
KCC 크롤링 중...현재 페이지 수 : 2
KCC 크롤링 중...현재 페이지 수 : 3
KCC 크롤링 중...현재 페이지 수 : 4
금호타이어 크롤링 중...현재 페이지 수 : 1
경동나비엔 크롤링 중...현재 페이지 수 : 1
원풍 크롤링 중...현재 페이지 수 : 1


## Highly relevant news extraction & title normalization

In [ ]:
# 기업명이 본문에 company_num 미만으로 나타나면 drop
def relevant_news_extraction(raw_data, company_name, company_num):
  p = company_name
  pattern = re.compile(p)
  for i, c in enumerate(raw_data['content']):
    if len(pattern.findall(str(c))) < company_num:
      raw_data.drop(i, inplace=True)
  return raw_data

In [ ]:
# 형태소 분석기 및 불용어 파일 불러오기
mecab = Mecab()
stopwords = []
with open('/content/drive/MyDrive/Colab Notebooks/data/korean_stopwords.txt', 'r', encoding='utf-8') as f:
    list_file = f.readlines()
    for word in list_file:
        stopwords.append(word.strip())  # 줄 끝의 줄 바꿈 문자를 제거한다.

In [ ]:
def preprocess(s):
# 정규화, 특수기호 제거
    def normalize(text):
        text=str(text).strip()
        text=re.compile('<.*?>').sub(' ', text) 
        text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
        text = re.sub('\s+', ' ', text)  
        text = re.sub(r'[0-9]',' ',text) 
        text=re.sub(r'[^\w\s]', ' ', str(text).strip())
        text = re.sub(r'\d',' ',text) 
        text = re.sub(r'\s+',' ',text) 
        return text

    # 명사/영단어 추출, 한글자 제외, 불용어 제거
    def word_filter(text):
        n = []
        word = mecab.nouns(text)  # 형태소 분석기를 바꿔가면서 해보기. 어차피 같은 konlpy 라이브러리이기 때문에, .nouns 앞에만 바꾸면 됨
        p = mecab.pos(text)
        for pos in p:
            if pos[1] in ['SL']:  # SL : 외국어
                word.append(pos[0])
        for w in word:
            if len(w)>1 and w not in stopwords:
                n.append(w)
        return " ".join(n)

    return word_filter(normalize(s))

In [ ]:
# 파일 읽어오기
for company in company_list:
    company_outputFile = company+outputFile
    data = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/data/craw_data/{company_outputFile}.csv')

    data.drop_duplicates(['title'], keep='first', inplace = True)  # 중복 기사 제거
    data.reset_index(inplace = True)  # 인덱스 초기화
    data = relevant_news_extraction(data, company, 5)
    data['title_noun'] = data['title'].apply(lambda x: preprocess(x))

    output_df = pd.DataFrame({
            'date' : data['date'],
            'title' : data['title'],
            'title_noun' : data['title_noun'],
            'content':data['content']
            })
        
    if output_df.empty:  # 이 부분은 해당 뉴스가 없을 때 바로 마지막 부분으로 향하는 코드 / 기업 리스트에서도 삭제
      print(f"{company}는(은) 해당 기간 뉴스가 없습니다.")
      output_df = pd.DataFrame({
        'date' : ['-'], 'label' : ['-'], 'score' : ['-'], 'title' : ['-'], 'content' : ['-'],
        'company' : company,
        'summary' : ["<해당 기간 뉴스가 없습니다.>"]
        })
      company_list.remove(company)
    output_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/craw_data/{company_outputFile}.csv', index=False)

In [ ]:
company_list

['KCC', '금호타이어', '경동나비엔']

## title clustering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# TF-IDF 임베딩
''' TF-IDF
 TF : 하나의 문서(문장)에서 특정 단어가 등장하는 횟수
 DF : 특정 단어가 몇 개의 문서(문장)에서 등장하는가. IDF는 그 역수
 최종 TF-IDF는 0~1사이
'''
def tfidf(data, ngram=5):
    if len(data) < 5:   # 데이터 수가 적으면 오류가 남
      m_num = 1
    else:
      m_num = 3
    tfidf_vectorizer = TfidfVectorizer(min_df = m_num, ngram_range=(1, ngram))  # TfidfVectorizer : min_df :최소 빈도값 (단어 등장하는 문서 수), ngram_range 단어 묶음 수
    tfidf_vectorizer.fit(data['title_noun']) #  text 데이터를 학습시켜줘야 한다. -> 학습이 끝나면 단어사전, IDF 벡터 출력 가능
    vector = tfidf_vectorizer.transform(data['title_noun']).toarray()  # 문서를 행렬화
    vector = np.array(vector)
    return vector

In [ ]:
# DBSCAN 클러스터링
'''
DBSCAN : 어느 점을 기준으로 반경(eps) 내에 점이 min_samples개 있다면 그 점을 core point라 판단 / 군집의 개수 k를 지정할 필요 없음
'''
def dbscan(company, data, eps, min_samples, metric = "cosine"):    # eps : 핵심 포인트를 중심으로 측정되는 유클리디언 거리값 / eps이 낮을수록, min_samples 값이 높을수록 군집으로 판단하는 기준이 까다로움.
    model = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)  # 거리 계산 식으로는 Cosine distance를 이용 /  ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan'] 등이 있음.
    vector_data = tfidf(data)
    result = model.fit_predict(vector_data)
    data['cluster'] = result
    no_clusterting = data[(data['cluster']==-1) | (data['cluster']==0)]  # -1, 0으로 클러스터링 된 데이터 제거
    no_clusterting.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/no_cluster/{company}_zero_cluster.csv')
    data.drop(no_clusterting.index, axis=0, inplace =True)
    data = order_cluster(data)
    return data

In [ ]:
# 최적의 K값 찾기
def silhouette_layer(data, param_init='random', param_n_init=10, param_max_iter=500):  
    data_length = len(data)
    clusters_range = range(2, data_length)
    results = []
    vector_data = tfidf(data)
    for i in clusters_range:
        # KMeans 클러스터링,  n_init : 초기 중심위치 시도 횟수. 디폴트는 10이고 10개의 무작위 중심위치 목록 중 가장 좋은 값을 선택한다. max_iter: 최대 반복 횟수.
        clusterer = KMeans(n_clusters=i, init=param_init, n_init=param_n_init, max_iter=param_max_iter, random_state=0)  
        cluster_labels = clusterer.fit_predict(vector_data)
        silhouette_avg = silhouette_score(vector_data, cluster_labels)  # 실루엣 스코어 계산
        results.append([i, silhouette_avg])
    result = pd.DataFrame(results, columns=["n_clusters", "silhouette_score"])

    if not result.empty:
      return int(result.loc[result["silhouette_score"].idxmax()]['n_clusters'])  # 실루엣 스코어가 최대인 n_cluster 반환
    else:
      return 1  # 비어있으면 클러스터 = 1로 통일

In [ ]:
# 군집별 기사 개수 확인 및 오름차순 정렬 
def order_cluster(data):
    counter = defaultdict(int)
    for c in data:
        counter[c] += 1
    sort_counter = sorted(counter.items(),  key=lambda item: item[1], reverse=True)
    print(dict(sort_counter))
    return data.sort_values(by='cluster')

# 기사 타이틀 확인
def print_cluster_result(data, result):
    # data : 데이터, result : 군집화 결과
    for cluster_num in set(result):
            temp_df = data[data['cluster'] == cluster_num] # cluster num 별로 조회
            print(f"군집 번호 : {cluster_num} / 뉴스 개수 : {len(temp_df)}")
            for title in temp_df['title']:
                print(title)
            print()

In [ ]:
for company in company_list:
    company_outputFile = company+outputFile
    data = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/data/craw_data/{company_outputFile}.csv')
    
    # DBSCAN 클러스터링
    data_db  = dbscan(company, data, eps=0.1, min_samples=2, metric = "cosine") 
    print_cluster_result(data_db, data_db['cluster'])
    data_db = data_db.drop_duplicates(['cluster'], keep='first') # 클러스터 당 대표 기사 1개 (첫번째 데이터 뽑아옴)
    
    # -1, 0으로 분류된 노이즈 데이터 K-means 클러스터링
    data_k =  pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/data/no_cluster/{company}_zero_cluster.csv')
    k = silhouette_layer(data_k)
    vector_k = tfidf(data_k, ngram=5)
    cluster_result = KMeans(n_clusters=k).fit_predict(vector_k)
    data_k['cluster'] = cluster_result
    data_k = data_k.drop_duplicates(['cluster'], keep='first')

    # 위의 두 데이터를 합치고 다시 한번 K-means 클러스터링, 대표 기사 하나 뽑기
    df = pd.concat([data_db,data_k])
    k_final = silhouette_layer(df)
    vector_3nd = tfidf(df, ngram=5)
    result_3nd = KMeans(n_clusters=k_final).fit_predict(vector_3nd)
    df['cluster'] = result_3nd
    df = df.drop_duplicates(['cluster'], keep='first') 
    df = order_cluster(df)
    df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/result_data/{company}_final.csv', index=False)

{'date': 1, 'title': 1, 'title_noun': 1, 'content': 1, 'cluster': 1}
군집 번호 : 1 / 뉴스 개수 : 2
KCC 페인트·창호, 소비자가 만든다…첫 프로슈머 모집
KCC, 프로슈머 모집…"페인트와 창호, 소비자가 만든다"

군집 번호 : 2 / 뉴스 개수 : 3
KCC, 소비자 의견 제품에 반영..프로슈머 마케팅 도입
KCC, 소비자와 소통 강화 위해 ‘프로슈머 마케팅’ 도입한다
"첫 프로슈머 돼주세요"…KCC 소비자 참여·체험 마케팅 도입

군집 번호 : 3 / 뉴스 개수 : 3
KCC, ‘동상동몽’ 건축도료 상생 세미나 개최
KCC, 전국 건축도료 대리점과 상생 세미나 개최
KCC, 대리점과 동반성장…건축도료 상생 세미나 개최

{'date': 1, 'title': 1, 'title_noun': 1, 'content': 1, 'cluster': 1, 'Unnamed: 0': 1}
{'date': 1, 'title': 1, 'title_noun': 1, 'content': 1, 'cluster': 1}


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1255: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1255: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


{'date': 1, 'title': 1, 'title_noun': 1, 'content': 1, 'cluster': 1, 'Unnamed: 0': 1}
{'date': 1, 'title': 1, 'title_noun': 1, 'content': 1, 'cluster': 1}
{'date': 1, 'title': 1, 'title_noun': 1, 'content': 1, 'cluster': 1, 'Unnamed: 0': 1}


In [ ]:
kcc =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/result_data/KCC_final.csv')
kcc

,date,title,title_noun,content,cluster,Unnamed: 0
0,2023-02-23,"KCC 페인트·창호, 소비자가 만든다…첫 프로슈머 모집",페인트 창호 소비자 프로슈머 모집 KCC,소비자 스킨십 강화를 위해 프로슈머 마케팅 도입1년간 프로슈머 활동 통해 얻은 소비...,0,NaN
1,2023-02-21,"KCC, ‘동상동몽’ 건축도료 상생 세미나 개최",동상 동몽 건축 도료 상생 세미나 개최 KCC,KCC 경영 가치 공유…신제품 특성·판매전략 설명세무·회계 실무 특강 등 대리점 동...,1,NaN
2,2023-02-22,"KCC, 소비자 의견 제품에 반영..프로슈머 마케팅 도입",소비자 의견 제품 반영 프로슈머 마케팅 도입 KCC,"KCC(대표 정몽진)가 마케팅 커뮤니케이션을 강화하고, 주요 제품 홍보 및 개발에 ...",2,NaN
3,2023-02-23,"'유리 1위' KCC글라스, 단일 제품으로만 여의도 면적 12배 판매",유리 글라스 단일 제품 여의도 면적 판매 KCC,친환경이 전방위적으로 핵심 키워드가 되며 유리 시장에서도 에너지 효율이 높은 고기능...,3,0.0


## sentiment classification

In [ ]:
kor_clf_sentiment = pipeline("sentiment-analysis", "snunlp/KR-FinBert-SC")

for company in company_list:
    company_outputFile = company+outputFile
    data = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/data/result_data/{company}_final.csv')
    test_dataset = data['title']

    output = kor_clf_sentiment(test_dataset.to_list())
    output_label = [info['label'] for info in output]
    output_score = [info['score'] for info in output]
    output_df = pd.DataFrame({
            'date' : data['date'],
            'label' : output_label,
            'score': output_score,
            'title' : data['title'],
            'title_noun' : data['title_noun'],
            'content':data['content']
            })
    
    output_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/result_data/{company}_final.csv', index=False)

## Keyword extraction

In [ ]:
# 키워드 추출
def keyword_extract(raw_data, ngram, top_n):
    result = []
    key_model = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')  
    title = list(raw_data['title_noun'])
    
    for i in title:
        if type(i) == str and len(i)>0:
            keyword = key_model.extract_keywords(i, keyphrase_ngram_range=(2,ngram), top_n=top_n)
            result.append(keyword[0][0])
        else:
            continue
    return result

# 빈도 수가 높은 키워드 확인
def most_keyword_select(keyword_result, most_num):
    keyword_split = [] ;kw = []
    for i in keyword_result:
        keyword_split.append(i.split(' '))
    for i in keyword_split:
        kw += i
    return Counter(kw).most_common(most_num)

In [ ]:
# 파일 읽어오기
from datetime import datetime
now = datetime.now()
company_list = ['SK하이닉스', '현대차', 'KCC', '현대차', '금호타이어', '경동나비엔']
outputFile = '_%s.%s.%s' % (now.year, now.month, now.day)

for company in company_list:
    company_outputFile = company+outputFile
    data = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/data/craw_data/{company_outputFile}.csv')
    keyword_result = keyword_extract(data, ngram = 5, top_n= 1)
    data = most_keyword_select(data, company, keyword_result, most_num=10, company_num=5)
    data.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/keyword_data/keyword_{company}.csv', index=False)

## text summarization

In [ ]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import numpy as np
import pandas as pd
import random
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("noahkim/KoT5_news_summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("noahkim/KoT5_news_summarization")

In [ ]:
import numpy as np
import random

def set_seed(random_seed):
    torch.random.manual_seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

In [ ]:
set_seed(777)

In [ ]:
# 후처리
def clean_text(inputString):
    inputString = re.sub('<pad>', '', inputString)
    inputString = re.sub(r'</s>', '', inputString)
    return inputString

# 요약
def summarize_text(text):
    input_ids = tokenizer.encode(text)
    gen_ids = model.generate(torch.tensor([input_ids]),
                            max_length=1000,repetition_penalty=1.4,
                            top_k=6,
                            temperature=1.0,                          
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id)
    generated = tokenizer.decode(gen_ids[0,:].tolist())
    generated = clean_text(generated)
    return generated

In [ ]:
# 파일 읽어오기
for company in company_list:
    output = pd.read_csv(f'.\\data\\result_data\\result_{company}_remove_duplicate.csv')
    content = output['content']
    summary = []
    
    if output.empty:
        summary.append('< 해당 뉴스 없음 >')
        output = pd.DataFrame({
        'date' : ['-'], 'label' : ['-'], 'score' : ['-'], 'title' : ['-'], 'content' : ['-'],
        'company' : '',
        'summary' : '',
    })
    else:
        for con in content:
            summary.append(summarize_text(con))

    output['summary'] = summary
    output['company'] = f'{company}'
    output = output[['date', 'label', 'score', 'company', 'title', 'content', 'summary']]
    output.to_csv(f'.\\data\\final\\summary_{company}.csv', index=False)
# %%
# 파일 합치기
company_0 = company_list[0]
df1 = pd.read_csv(f'.\\data\\final\\summary_{company_0}.csv')
for company in company_list[1:]:
    df2 = pd.read_csv(f'.\\data\\final\\summary_{company}.csv')
    df1 = pd.concat([df1,df2]) 
df1.to_csv(f'.\\data\\final\\summary_final.csv', index=False)